In [1]:
from keras.utils import to_categorical
import keras

Using TensorFlow backend.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
plt.close()

# Set the directories for the data and the CSV files that contain ids/labels
dir_train_images  = './training/'
dir_test_images   = './testing/'
dir_train_labels  = './labels_training.csv'
dir_test_ids      = './sample_submission.csv'

In [3]:
def load_data(dir_data, dir_labels, training=True):
    ''' Load each of the image files into memory 

    While this is feasible with a smaller dataset, for larger datasets,
    not all the images would be able to be loaded into memory

    When training=True, the labels are also loaded
    '''
    labels_pd = pd.read_csv(dir_labels)
    ids       = labels_pd.id.values
    data      = []
    for identifier in ids:
        fname     = dir_data + identifier.astype(str) + '.tif'
        image     = mpl.image.imread(fname)
        data.append(image)
    data = np.array(data) # Convert to Numpy array
    if training:
        labels = labels_pd.label.values
        return data, labels
    else:
        return data, ids

In [4]:
data, y_binary = load_data(dir_train_images, dir_train_labels, training=True)

test_data, ids = load_data(dir_test_images, dir_test_ids, training=False)

img_rows, img_cols = 101, 101

data = data.astype('float32')
data /= 255

test_data = test_data.astype('float32')
test_data /= 255

input_shape = img_rows, img_cols, 3

In [5]:
train_data, validation_data, train_label, validation_label = train_test_split(data, y_binary, test_size=100, random_state=114)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = input_shape, activation = 'relu'))

# Step 2 - Pooling
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(0.2))

# Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(0.2))

# Adding a third convolutional layer
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.2))

# Adding a fourth convolutional layer
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(0.2))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(loss=keras.losses.binary_crossentropy,
          optimizer=keras.optimizers.Adam(),
          metrics=['accuracy'])

In [7]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 99, 99, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 128)      

In [8]:
history4 = classifier.fit(
    data, 
    y_binary,
    batch_size=100,
    epochs=40,
    verbose=1,
    validation_data=(validation_data, validation_label))

Train on 1500 samples, validate on 100 samples
Epoch 1/40
1500/1500 [==============================] - 26s 17ms/step - loss: 0.6451 - accuracy: 0.6513 - val_loss: 0.6701 - val_accuracy: 0.6200
Epoch 2/40
1500/1500 [==============================] - 25s 17ms/step - loss: 0.6600 - accuracy: 0.6633 - val_loss: 0.6677 - val_accuracy: 0.6200
Epoch 3/40
1500/1500 [==============================] - 28s 18ms/step - loss: 0.6407 - accuracy: 0.6633 - val_loss: 0.6682 - val_accuracy: 0.6200
Epoch 4/40
1500/1500 [==============================] - 26s 17ms/step - loss: 0.6412 - accuracy: 0.6633 - val_loss: 0.6729 - val_accuracy: 0.6200
Epoch 5/40
1500/1500 [==============================] - 26s 17ms/step - loss: 0.6354 - accuracy: 0.6633 - val_loss: 0.6669 - val_accuracy: 0.6200
Epoch 6/40
1500/1500 [==============================] - 25s 17ms/step - loss: 0.6189 - accuracy: 0.6647 - val_loss: 0.6587 - val_accuracy: 0.6600
Epoch 7/40
1500/1500 [==============================] - 26s 18ms/step - loss:

In [9]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=90, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)

batches = gen.flow(data, y_binary, batch_size=100)
validation_batches = gen.flow(validation_data, validation_label, batch_size=100)

history4 = classifier.fit_generator(batches, epochs=150, validation_data=validation_batches)

Epoch 1/150
15/15 [==============================] - 14s 937ms/step - loss: 0.4014 - accuracy: 0.8433 - val_loss: 0.4109 - val_accuracy: 0.8200
Epoch 2/150
15/15 [==============================] - 14s 924ms/step - loss: 0.3646 - accuracy: 0.8500 - val_loss: 0.4797 - val_accuracy: 0.7500
Epoch 3/150
15/15 [==============================] - 12s 821ms/step - loss: 0.3389 - accuracy: 0.8640 - val_loss: 0.3303 - val_accuracy: 0.8600
Epoch 4/150
15/15 [==============================] - 12s 829ms/step - loss: 0.3599 - accuracy: 0.8560 - val_loss: 0.4036 - val_accuracy: 0.8600
Epoch 5/150
15/15 [==============================] - 14s 920ms/step - loss: 0.3075 - accuracy: 0.8807 - val_loss: 0.3215 - val_accuracy: 0.8800
Epoch 6/150
15/15 [==============================] - 14s 953ms/step - loss: 0.3000 - accuracy: 0.8860 - val_loss: 0.2587 - val_accuracy: 0.8900
Epoch 7/150
15/15 [==============================] - 15s 978ms/step - loss: 0.3244 - accuracy: 0.8653 - val_loss: 0.3280 - val_accuracy:

In [10]:
r = classifier.predict(data)

In [11]:
import numpy
numpy.mean(r)

0.33345085

In [12]:
r

array([[6.0356327e-04],
       [1.1066999e-03],
       [5.5460244e-01],
       ...,
       [4.3951950e-05],
       [1.5337758e-02],
       [9.9711072e-01]], dtype=float32)

In [13]:
y_binary

array([0, 0, 1, ..., 0, 0, 1], dtype=int64)

In [14]:
numpy.sum(numpy.abs((r>0.5).reshape((1500,)) - y_binary)) / 1500

0.022

In [15]:
produce_submission = True # Switch this to True when you're ready to create a submission for Kaggle

test_scores    = classifier.predict(test_data)
test_scores = test_scores.reshape((test_scores.shape[0],))
# Save the predictions to a CSV file for upload to Kaggle
submission_file = pd.DataFrame({'id':    ids,
                               'score':  test_scores})

In [16]:
if produce_submission:
    submission_file.to_csv('yyf_simple_cnn_4_layers.csv',
                           columns=['id','score'],
                           index=False)

In [17]:
submission_file.head()

,id,score
0,1500,0.012872
1,1501,0.000333
2,1502,0.957085
3,1503,0.078518
4,1504,0.000051
